# Finetuning for RAG

Finetuning is a technique that consists of taking a pretrained or "frozen" model and adapting it to the current context by training it with a datasat based on the knowledge base that the model needs to answer queries about.

Finetuning is sometimes used instead of RAG, but it can also be used in conjunction with RAG to improve the performance of the model and is often what can give the last bit of performance, when build RAG pipelines that are to be used in production.  When you finetune for RAG you have multiple different components which can be finetuned for different tasks:

- **Indexing**: Fintetuning the *embedding* model for higher similarity between queries and their relevant documents
- **Pre-retrieval**: Finetune LLMs used in *query routing* or *query-rewriting*.
- **Retriever**: Finetune LLMs used in *retrieval* like for *iterative*, *recursive* or *generative* retrieval.
- **Post-retrieval**: Finetuning your *reranking* model or prompt *compressor*
- **Generator**: If you are using a generator model, you can finetune it to better generate the answers to the queries.

In many cases, it makes sense to finetune the different components of the RAG pipeline separately, as they are often trained on different datasets and have different objectives.

In [ ]:
%pip install llama-index-finetuning spacy

In [ ]:
import os
from dotenv import load_dotenv
from util.helpers import get_malazan_pages, create_and_save_md_files

from IPython.display import display, Markdown

from llama_index.core.evaluation import EmbeddingQAFinetuneDataset
from llama_index.finetuning import generate_qa_embedding_pairs, generate_cohere_reranker_finetuning_dataset
from llama_index.finetuning.callbacks import OpenAIFineTuningHandler
from llama_index.core import SimpleDirectoryReader
from llama_index.llms.openai import OpenAI

from llama_index.core.llama_dataset.generator import RagDatasetGenerator

In [ ]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes.
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.
import nest_asyncio
nest_asyncio.apply()

Add the following to a `.env` file in the root of the project if not already there.

```
OPENAI_API_KEY=<YOUR_KEY_HERE>
```

In [ ]:
load_dotenv(override=True, verbose=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
llm = OpenAI(api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")

In [ ]:
pages = get_malazan_pages(["Anomander Rake"])
create_and_save_md_files(pages, path="./data/docs/finetune/")
documents = SimpleDirectoryReader("./data/docs/finetune").load_data()

## Automatic training data generation

TODO

In [ ]:
question_gen_query = (
    "You are a Teacher/ Professor. Your task is to setup a quiz/examination."
    "Using the provided context, formulate a single question that captures an important fact from the context."
    "Restrict the question to the context information provided."
)

dataset_generator = RagDatasetGenerator.from_documents(
    documents,
    question_gen_query=question_gen_query,
    llm=llm,
)
questions = dataset_generator.generate_dataset_from_nodes()

In [ ]:
text = "\n\n--".join([question.query for question in questions.examples[5:10]])
display(Markdown(f'--{text}'))

## Fine-tune embeddings

https://docs.llamaindex.ai/en/stable/examples/finetuning/embeddings/finetune_embedding/

## Fine-tune reranker

https://docs.llamaindex.ai/en/stable/examples/finetuning/rerankers/cohere_custom_reranker/

## Fine-tune generator

https://docs.llamaindex.ai/en/stable/examples/finetuning/openai_fine_tuning/